<a href="https://colab.research.google.com/github/francji1/01ZLMA/blob/main/R/01ZLMA_ex10_Poisson.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 01ZLMA - Exercise 10
Exercise 10 of the course 01ZLMA. 

## Contents

* Log-linear models with Poisson distributed data
 ---
* Example from the Lecture 10 (Section 7.7. from lecture notes)
 ---
* ..



## Poission regression

Poission regression is good for modeling random variables expressing the number of occurs of independent events in given time period.  
It also proves to be more suitable for binomial data if the number of repetitions is large and the probability of success low.


*   $Y_1, \ldots, Y_n$ $iid$ $Y_i \sim Po(\mu_i)$, where $\mu_i = s_i λ_i$ and $s_i$ is known sample size.
*   Canonical link function $g(x) = ln(x)$: $\eta_i = g(\mu_i) = ln(\mu_i) = ln(s_i) + ln(\lambda_i) = ln(s_i) + x_i^T \beta$, where $i= 1,\ldots, n$ and $\mu_i = E[Y_i] = s_i\lambda_i = s_i e^{x_i^T \beta}$.




Let us assume exercise 7.7 from lecture notes (Dobson 9.2.1) - British doctors' smoking and coronary death. (https://reneues.files.wordpress.com/2010/01/an-introduction-to-generalized-linear-models-second-edition-dobson.pdf)

Data from the famous doctors study of smoking conducted by Sir Richard Doll and colleagues.

---



In [ ]:
data_poiss <- "https://raw.githubusercontent.com/francji1/01ZLMA/main/data/smoke.csv"
data_poiss  <- read.table(data_poiss, header = T, sep = ";")
data_poiss

In [ ]:
library(tidyverse)
library(MASS)
install.packages("dobson")
library(dobson)
data(doctors)
? doctors

In [ ]:

df <- data_poiss %>%
 add_column(age_avg = c(40,50,60,70,80,40,50,60,70,80),
            age_min = c(35,45,55,65,75,35,45,55,65,75)) %>%
 mutate(n_min = ceiling(person_years/(age_min-10)),
        n_avg = ceiling(person_years/(age_avg-10)),
        living = n_min - deaths,
        death_rate= deaths/(person_years/10000))
 str(df)
 df

Add a death rate: the number of deaths per 10,000 person years:
$$ \text{death_rate} = \frac{deaths}{\frac{\text{person_years}}{10000}}$$


In [ ]:
attach(df)

In [ ]:
is.factor(smoke)
smoke <- as.factor(smoke)
smoke

In [ ]:
agecat <- as.numeric(as.factor(age_group))
agecat

## Nejjednodušší aditivní model 
Začneme nejjednodušším modelem s proměnnými `smoke` a `agecat`

In [ ]:
mdl <- glm(deaths ~ offset(log(person_years)) + smoke + agecat, family=poisson(link="log"))
summary(mdl)

Deviační statistika má obrovskou hodnotu a model tak moc neodpovídá datům.
Zobrazíme závislost logaritmu škálovaných hodnot Y na proměnné agecat

In [ ]:
y_scaled <- deaths/person_years*100000
plot(agecat, log(y_scaled), pch = c(15:16)[smoke], col = c("red","blue")[smoke], 
     cex=1.2, xlab = "agecat", ylab = "počty úmrtí")

Je jasně vidět, že vztah není lineární, přidáme do modelu proměnnou `agecat^2`

## Model s kvadratickou závislostí na věku - Model 0  

In [ ]:
mdl_0 <- glm(deaths ~ offset(log(person_years)) + smoke + agecat + I(agecat^2), 
              family=poisson(link="log"))
summary(mdl_0)

Všechny regresory jsou v modelu významné a hodnota deviační statistiky klesla na 12.176, porovnejme ji s kritickou hodnotou LRT testu

In [ ]:
Xm <- model.matrix(mdl_0)
n  <- nrow(Xm)
p  <- ncol(Xm)
Xm;n; p
# kritická hodnota
c_val <- qchisq(0.05, n-p, ncp=0, lower.tail = FALSE); 
c_val
# p-hodnota testu
p_val <- pchisq(deviance(mdl_0), n-p, ncp=0, lower.tail = FALSE); 
p_val

Hypotézu, že model dobře popisuje data jsme na hladině významnosti 5% těsně nezamítli. Zkusíme ještě přidat do modelu interakci (na obrázku z přednášky byla vidět možná změna závislosti na kouření s rostoucím věkem)

## Model s kvadratickou závislostí na věku a interakcí - Model 1 

In [ ]:
mdl_1 <- glm(deaths ~ offset(log(person_years)) + smoke + agecat + I(agecat^2) + smoke:agecat, 
              family=poisson(link="log"))
summary(mdl_1)

Všechny proměnné jsou v modelu významné a hodnota deviační statsitiky klesla na hodnotu 1.6354. 
Pro kritickou hodnotu LRT testu dostaneme

In [ ]:
Xm <- model.matrix(mdl_1)
n  <- nrow(Xm);
p  <- ncol(Xm);
n; p
# kritická hodnota
c_val <- qchisq(0.05, n-p, ncp=0, lower.tail = FALSE);
c_val
# p-hodnota testu
p_val <- pchisq(deviance(mdl_1), n-p, ncp=0, lower.tail = FALSE);
p_val

a hypotézu vhodnosti modelu tedy nezamítáme s p-hodnotou 0.897. To indikuje, že model dobře popisuje data.

Pro ilustraci ještě spočtěme hodnotu deviační statistiky pomocí vzorce z přednášky

LRT Test for Poiison regression by Deviance sttistics:
$$S = 2\sum_{i=1}^n \left[ y_i (ln(\frac{y_i}{s_i}) - x_i^T \hat{\beta}) - s_i (\frac{y_i}{s_i} - e^{x_i^T \hat{\beta}}) )\right] = 2\sum_{i=1}^n \left[ y_i ln(\frac{y_i}{\hat{\mu_i}}) - (y_i - \hat{\mu_i})\right]
$$

In [ ]:
y <- deaths
mu_hat <- fitted(mdl_1)               # predikované hodnoty modelem 1
dev <- y*log(y/mu_hat) - (y - mu_hat)
dev_stat <- 2*sum(dev); 
dev_stat

Dostáváme stejnou hodnotu jako pomocí funkce `glm()`.

Ještě otestujeme významnost interakce pomocí vzorce odvozeného na přednášce

In [ ]:
mu_tilde <- fitted(mdl_0)            # predikované hodnoty modelem 0
dev <- y * log(mu_hat/mu_tilde) - (mu_hat - mu_tilde)
dev_stat <- 2*sum(dev);
dev_stat     # deviační statistika
deviance(mdl_0) - deviance(mdl_1)    # pro kontrolu pomocí naladěných modelů

pro kritickou hodnotu testu platí 

In [ ]:
# kritická hodnota
c_val <- qchisq(0.05, 1, ncp=0, lower.tail = FALSE)
c_val
# p-hodnota testu
p_val<-pchisq(dev_stat, 1, ncp=0, lower.tail = FALSE) 
p_val

interakce je tedy v modelu významná. Případně můžeme použít přímo funkci `anova()`

In [ ]:
anova(mdl_0,mdl_1, test="Chisq")

která vrací stejný výsledek.


## Analýza Modelu 1

Scatterplot pro pozorované a predikované hodnoty vysvětlované proměnné

In [ ]:
plot(y,mu_hat,col=2, cex=1.0, lwd=2)
abline(1,1,col=4,lwd=2)

Pozorované a predikované hodnoty velmi dobře odpovídají.

**Obrázky reziduí**

In [ ]:
par(mfrow=c(2,2))

r_ps <- rstandard(mdl_1, type="pearson") # standardizovaná Pearsonova rezidua

# rezidua vs. fitted values
plot(mu_hat,r_ps,col="red", cex=1.3, lwd=2)
abline(0,0)
# rezizua vs. agecat
plot(agecat,r_ps,col="red", cex=1.3, lwd=2)
abline(0,0)
# rezizua vs. smoke
plot(smoke,r_ps,col="red", cex=1.3, xlab="Smoke", lwd=2)
abline(0,0)

qqnorm(r_ps)   # QQ - plot reziduí
qqline(r_ps)

vzhledem k mlému počtu pozorování zde není vidět žádný zásadní problé. Otestujme ještě formálně normalitu rezidí

In [ ]:
shapiro.test(r_ps)  # Test normality

Hypotéza normality není zamítnuta.

**Influenční pozorování a pákové body**

In [ ]:
par(mfrow=c(1,2))
c_d <- cooks.distance(mdl_1)  # Cookova vzdálenost
lev <- hatvalues(mdl_1)       # diag. prvky projekční matice

plot(c_d,col="red", cex=1.3, lwd=2, ylim = c(0,2.1))
# abline(8/(n-2*p),0)         # v našem případě máme malý počet pozorování a n-2p = 0
                              # kritická hodnota by tedy byla velká

plot(lev,col="red", cex=1.3, lwd=2, ylim = c(0,1.2))
abline(2*p/n,0)

Ani v jednom případě nevidíme podezřele velké hodnoty

## Interpretace parametrů

Spočteme relativní rizika v Modelu 1

In [ ]:
par_est_1 <- coef(mdl_1);
par_est_1
RR_1 <- exp(par_est_1)
RR_1

Diskuze obdržených hodnot a výpočet relativních rizik pro jednotlivé kategorie věku jsou v Sekci 7.7 materiálů k přednášce.

Pro Model 0 bez interakcí je situace jednodušší a lze snado získat odhady RR a příslušné intervaly spolehlivosti

In [ ]:
exp(cbind(coef(mdl_0), confint(mdl_0)))

**Na závěr ještě vykresleme obrázky Modelů 0 a 1**

In [ ]:
par(mfrow=c(2,1))

x_pop<-age_group[1:5]
s <- person_years
y_scaled <- y/s*100000
smoke <- ordered(smoke, levels = c("1","0"))   # udáme pořadí úrovní faktoru Smoke, `R` je jinak prohodí, což může způsobit špatnou legendu v grafech

### Obrázek model 0
mu_tilde_scaled <- mu_tilde/s*100000

plot(agecat, y_scaled, pch = c(15:16)[Smoke],  xaxt = "n", main = "Model 0 bez interakce", col = c("red","blue")[smoke], cex=1.0, xlab = "Age group", ylab = "Počty úmrtí")
axis(1, at=agecat[1:5], labels=x_pop)
legend("topleft", inset = .05, bty="n", legend = c("Smokers", "Non-smokers"), pch = c(15:16), cex=1.0, pt.cex = 1.0, col = c("red","blue"))
lines(agecat[1:5], mu_tilde_scaled[1:5], col="red", lwd=2 )
lines(agecat[6:10], mu_tilde_scaled[6:10], col = "blue", lwd=2 )


### Obrázek model 1
mu_hat_scaled <- mu_hat/s*100000

plot(agecat, y_scaled, pch = c(15:16)[smoke],  xaxt = "n", main = "Model 1 s interakcí", col = c("red","blue")[smoke], cex=1.0, xlab = "Age group", ylab = "Počty úmrtí")
axis(1, at=agecat[1:5], labels=x_pop)
legend("topleft", inset = .05, bty="n", legend = c("Smokers", "Non-smokers"), pch = c(15:16), cex=1.0, pt.cex = 1.0, col = c("red","blue"))
lines(agecat[1:5], mu_hat_scaled[1:5], col="red", lwd=2 )
lines(agecat[6:10], mu_hat_scaled[6:10], col = "blue", lwd=2 )


### Task: 
* Try to model with and without offset. Compare results, why to use/ not use offset?
* Try Poisson distribution with factor variables (contingency tables approach).

